In [1]:
# ensemble strandedness

In [2]:
neg='/home/hsher/scratch/circ_nextera_iter13/output/edits/APOBEC1only.dp4.neg.vcf.aggregated.nonzero.snpfiltered.edit_score.tsv'
pos='/home/hsher/scratch/circ_nextera_iter13/output/edits/APOBEC1only.dp4.pos.vcf.aggregated.nonzero.snpfiltered.edit_score.tsv'

In [17]:
# according to gtf, pick the relevant strand
gtf = '/home/hsher/scratch/circ_nextera_iter13/output/APOBEC1only.gtf'
outf = '/home/hsher/scratch/circ_nextera_iter13/output/edits/APOBEC1only.final_edit_score.tsv'

In [4]:
import pandas as pd
from pybedtools import BedTool
def read_ciri_gtf(fname):
    bed = BedTool(fname)
    df = bed.to_dataframe()
    # filter for non-entries
    stat = df.loc[df['seqname'].str.contains('##'), 'seqname'].str.split(': ', expand = True)
    df = df.loc[~df['seqname'].str.contains('##')].reset_index()
    
    # get attributes
    attrs = pd.DataFrame([i.attrs for i in bed])
    
    return pd.concat([df, attrs], axis = 1), stat

In [5]:
neg_df = pd.read_csv(neg, sep = '\t', index_col = 0)
pos_df = pd.read_csv(pos, sep = '\t', index_col = 0)
circ_anno, stat = read_ciri_gtf(gtf)

In [6]:
neg_df['POS']=neg_df['pos_id'].apply(lambda s: int(s.split(':')[-1]))
neg_df['circ_id']=neg_df['pos_id'].apply(lambda s: ':'.join(s.split(':')[:-1]))

pos_df['POS']=pos_df['pos_id'].apply(lambda s: int(s.split(':')[-1]))
pos_df['circ_id']=pos_df['pos_id'].apply(lambda s: ':'.join(s.split(':')[:-1]))

neg_df['strand']='-'
pos_df['strand']='+'

In [7]:
circ_anno['strand'].value_counts()

+    28897
-    28317
Name: strand, dtype: int64

In [8]:
pos_circ = circ_anno.loc[circ_anno['strand']=='+', 'circ_id']
neg_circ = circ_anno.loc[circ_anno['strand']=='-', 'circ_id']

In [9]:
pos_df = pos_df.loc[pos_df['circ_id'].isin(pos_circ)]
neg_df = neg_df.loc[neg_df['circ_id'].isin(neg_circ)]

In [10]:
edit_df = pd.concat([pos_df, neg_df])

In [11]:
edit_df

,pos_id,n_ref,n_alt,total,p_hetero,p_homo,edit_frac,theta,confidence,pvalue,FDR,POS,circ_id,strand
2,chr1:62439799|62445429:5074,28,2,30,4.051253e-07,4.263435e-54,0.066667,0.066667,0.570785,4.292155e-01,1.000000e+00,5074,chr1:62439799|62445429,+
4,chr1:220791491|220805274:6392,28,2,30,4.051253e-07,4.263435e-54,0.066667,0.066667,0.570785,4.292155e-01,1.000000e+00,6392,chr1:220791491|220805274,+
5,chr1:51760690|51765981:5247,28,2,30,4.051253e-07,4.263435e-54,0.066667,0.066667,0.570785,4.292155e-01,1.000000e+00,5247,chr1:51760690|51765981,+
6,chr1:75732644|75733628:243,28,2,30,4.051253e-07,4.263435e-54,0.066667,0.066667,0.570785,4.292155e-01,1.000000e+00,243,chr1:75732644|75733628,+
7,chr1:115651277|115664268:160,28,2,30,4.051253e-07,4.263435e-54,0.066667,0.066667,0.570785,4.292155e-01,1.000000e+00,160,chr1:115651277|115664268,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226143,chrX:53614534|53615835:1224,269,11,280,8.765753e-66,0.000000e+00,0.039286,0.039286,0.172239,8.277615e-01,1.000000e+00,1224,chrX:53614534|53615835,-
226145,chrX:44524002|44538542:270,254,9,263,9.746569e-64,0.000000e+00,0.034221,0.034221,0.089607,9.103931e-01,1.000000e+00,270,chrX:44524002|44538542,-
226147,chrX:73826738|73841635:2468,392,10,402,2.626858e-102,0.000000e+00,0.024876,0.024876,0.004103,9.958971e-01,1.000000e+00,2468,chrX:73826738|73841635,-
226148,chrX:44524002|44527326:270,183,36,219,2.735081e-25,0.000000e+00,0.164384,0.164384,1.000000,3.324129e-10,5.573925e-08,270,chrX:44524002|44527326,-


In [14]:
from statsmodels.stats.multitest import fdrcorrection
_,edit_df['FDR'] = fdrcorrection(edit_df['pvalue'])


In [18]:
edit_df.to_csv(outf, sep = '\t')